In [1]:
import pandas as pd
import os
import json

In [2]:
def clean_dataframe(path_1, path_2):
    with open(path_1, 'r') as file:
        data = json.load(file)

    flattened_data = []
    for key, value in data.items():
        row = {'id_EXIST': key}
        for k, v in value.items():
            row[k] = v
        flattened_data.append(row)

    df = pd.DataFrame(flattened_data)

    with open(path_2, 'r') as file:
        data = json.load(file)

    gold = pd.DataFrame(data)
    df = pd.merge(df, gold, left_on='id_EXIST', right_on='id')

    df = df[['id','tweet','value']]
    return df

In [3]:
dpath_training = '/kaggle/input/exist-2024-task1/EXIST2024_training.json'
dpath_gold = '/kaggle/input/exist-2024-task1/EXIST2024_training_task1_gold_hard.json'
df = clean_dataframe(dpath_training, dpath_gold)
df.head()

,id,tweet,value
0,100001,"@TheChiflis Ignora al otro, es un capullo.El p...",YES
1,100002,@ultimonomada_ Si comicsgate se parece en algo...,NO
2,100003,"@Steven2897 Lee sobre Gamergate, y como eso ha...",NO
3,100005,@novadragon21 @icep4ck @TvDannyZ Entonces como...,YES
4,100006,@yonkykong Aaah sí. Andrew Dobson. El que se d...,NO


In [4]:
dpath_dev = '/kaggle/input/exist-2024-task1/EXIST2024_dev.json'
dpath_gold_dev = '/kaggle/input/exist-2024-task1/EXIST2024_dev_task1_gold_hard.json'
df_dev = clean_dataframe(dpath_dev, dpath_gold_dev)
df_dev.head()

,id,tweet,value
0,300002,@anacaotica88 @MordorLivin No me acuerdo de lo...,YES
1,300003,@cosmicJunkBot lo digo cada pocos dias y lo re...,NO
2,300004,Also mientras les decia eso la señalaba y deci...,YES
3,300005,"And all people killed, attacked, harassed by ...",NO
4,300006,On this #WorldPressFreedomDay I’m thinking of ...,NO


In [5]:
df['value'] = df['value'].map({'YES':1,'NO':0})
df_dev['value'] = df_dev['value'].map({'YES':1,'NO':0})

In [6]:
# Function to remove words starting with '@'
# def remove_mentions(text):
#     words = text.split()
#     filtered_words = [word for word in words if not word.startswith('@')]
#     return ' '.join(filtered_words)

# # Apply the function to the 'tweet' column
# df['tweet'] = df['tweet'].apply(remove_mentions)
# df_dev['tweet'] = df_dev['tweet'].apply(remove_mentions)
# df.head()

In [9]:
pip install es-core-news-sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 76.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 30.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.5/659.5 kB 29.3 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.5.3
    Uninstalling pydantic-2.5.3:
      Successfully uninstalled pydantic-2.5.3
  Attempting uninstall: thinc
    Found ex

In [10]:
import spacy
nlp = spacy.load("es_core_news_sm")
unique_words = set()

def remove_punct(text):
    doc = nlp(text)
    entry = ""
    for token in doc:
        if token.pos_ not in ["SPACE", "X", "PUNCT", "AUX"] and not token.is_stop:
            unique_words.add(token.lemma_)
            entry += " "
            entry += token.lemma_
    return entry

df['tweet'] = df['tweet'].apply(remove_punct)
df_dev['tweet'] = df_dev['tweet'].apply(remove_punct)

/opt/conda/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'es_core_news_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [11]:
len(unique_words)

39613

In [13]:
import math

def calculate_tf_idf(sentence):
    idf_denom = [0] * len(unique_words)
    tf_denom = 0
    tf_nom = []
    idf_nom = 1 + len(sentence)
    for j, word in enumerate(unique_words):
        frequency = sentence.count(word)
        tf_nom.append(frequency)
        tf_denom += frequency
        if frequency != 0:
            idf_denom[j] +=1
    return [tf_nom[k]/tf_denom * math.log(idf_nom/(idf_denom[k]+1)) for k in range(len(unique_words))]

df['tfidf'] = [None] * len(df)
df_dev['tfidf'] = [None] * len(df_dev)

df['tfidf'] = df.tweet.apply(calculate_tf_idf)
df_dev['tfidf'] = df_dev.tweet.apply(calculate_tf_idf)

In [14]:
df.head()

,id,tweet,value,tfidf
0,100001,@TheChiflis Ignora capullo problema youtuber ...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,100002,@ultimonomado _ comicsgate a gamergate acoso ...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,100003,@steven2897 Lee Gamergate y cambiar comunicar...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,100005,@novadragon21 @icep4ck @tvdannyz mercado camb...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,100006,@yonkykong Aaah Andrew Dobson dedicar a echar...,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [15]:
X = df['tfidf']
X = [list(map(float, x)) for x in X]
y = df['value']

X_val = df_dev['tfidf']
X_val = [list(map(float, x)) for x in X_val]
y_val = df_dev['value']

In [16]:
# del unique_words
# del df
# del df_dev

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model = RandomForestClassifier()
model.fit(X, y)
predictions = model.predict(X_val)
accuracy = accuracy_score(y_val, predictions)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_val, predictions, digits=4))
conf_matrix = confusion_matrix(y_val, predictions)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.68
Classification Report:
              precision    recall  f1-score   support

           0     0.6361    0.8831    0.7395       479
           1     0.7918    0.4681    0.5884       455

    accuracy                         0.6809       934
   macro avg     0.7140    0.6756    0.6640       934
weighted avg     0.7120    0.6809    0.6659       934

Confusion Matrix:
[[423  56]
 [242 213]]


In [18]:
from sklearn.svm import SVC

svm_model = SVC()
svm_model.fit(X, y)
svm_predictions = svm_model.predict(X_val)
svm_accuracy = accuracy_score(y_val, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.2f}")
print("SVM Classification Report:")
print(classification_report(y_val, svm_predictions, digits=4))
svm_conf_matrix = confusion_matrix(y_val, svm_predictions)
print("SVM Confusion Matrix:")
print(svm_conf_matrix)

SVM Accuracy: 0.71
SVM Classification Report:
              precision    recall  f1-score   support

           0     0.6625    0.8810    0.7563       479
           1     0.8081    0.5275    0.6383       455

    accuracy                         0.7088       934
   macro avg     0.7353    0.7042    0.6973       934
weighted avg     0.7334    0.7088    0.6988       934

SVM Confusion Matrix:
[[422  57]
 [215 240]]


In [19]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier()
xgb_model.fit(X, y)
xgb_predictions = xgb_model.predict(X_val)
xgb_accuracy = accuracy_score(y_val, xgb_predictions)
print(f"XGBoost Accuracy: {xgb_accuracy:.2f}")
print("XGBoost Classification Report:")
print(classification_report(y_val, xgb_predictions, digits=4))
xgb_conf_matrix = confusion_matrix(y_val, xgb_predictions)
print("XGBoost Confusion Matrix:")
print(xgb_conf_matrix)

XGBoost Accuracy: 0.77
XGBoost Classification Report:
              precision    recall  f1-score   support

           0     0.7330    0.8539    0.7888       479
           1     0.8138    0.6725    0.7365       455

    accuracy                         0.7655       934
   macro avg     0.7734    0.7632    0.7626       934
weighted avg     0.7724    0.7655    0.7633       934

XGBoost Confusion Matrix:
[[409  70]
 [149 306]]
